## Mahyar Mohammadi Matin - 610398166 - HW2

**States:** A state in the 8-puzzle problem is a configuration of the tiles on the puzzle. Each state is represented as a 3x3 numpy array numbered from 0 to 8, where 0 represents the blank tile.

**Operators (Actions):** The operators, or actions, in the 8-puzzle problem are the moves that can be made to change the state of the puzzle. In this problem, the actions are moving the blank tile up, down, left, or right.

**Goal Test:** The goal test in the 8-puzzle problem is to determine whether the puzzle has been solved, which means that the tiles are in their correct place, numbered from 1 to 8 with 0 as the last.

**Path Cost:** The path cost in the 8-puzzle problem is the number of moves taken to reach the goal state from the initial state. In this problem, the path cost is a simple count of the number of actions taken.


**Transition Model:** The transition model in the 8-puzzle problem describes how the state of the puzzle changes as a result of an action. In this problem, the transition model is implemented in the move function, which takes in the current state of the puzzle and the action (direction to move the blank tile), and returns the new state of the puzzle after making that move.

**Best huristic function:** The best heuristic function for the 8-puzzle problem would be the Manhattan distance heuristic. It calculates the sum of the distances of each tile from its goal position. The Manhattan distance of a tile is the sum of the horizontal and vertical distances of the tile from its goal position. This heuristic has been proven to be admissible, meaning that it never overestimates the actual cost, and consistent, meaning that the cost of moving from one tile to an adjacent tile is always less than or equal to the sum of the heuristic values of the two tiles. These properties guarantee that the A* search algorithm, which uses the heuristic function, will always find the optimal solution.

but to be admissible I remove distance of zero 

In [59]:
import heapq
import time
import math
import numpy as np
import pandas as pd
import copy
from collections import deque
from queue import PriorityQueue

### check if the problem is actualy solvable

In [1]:
def is_solvable(start, goal):
    start_flat = start.flatten()
    goal_flat = goal.flatten()
    start_inversions = 0
    goal_inversions = 0
    for i in range(len(start_flat)):
        for j in range(i, len(start_flat)):
            if start_flat[i] and start_flat[j] and start_flat[i] > start_flat[j]:
                start_inversions += 1
            if goal_flat[i] and goal_flat[j] and goal_flat[i] > goal_flat[j]:
                goal_inversions += 1
    return (start_inversions % 2) == (goal_inversions % 2)

In [74]:
class State:
    def __init__(self, data, parent, move):
        self.data = data
        self.parent = parent
        self.move = move
        self.row, self.col = np.where(data == 0)
        self.row, self.col = self.row[0], self.col[0]
        
class Algorithms:
    def __init__(self,algo_names=['A*','UCS','IDS','BFS','DFS']):
        self.names = algo_names
        self.memory = 0
        self.start_time = time.time()
        
    def run(self,algo_name,start,goal=None):
        if goal is None:
            goal = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 0]])
        if not is_solvable(start, goal):
            print('is not solvable')
            return None,0,0
            
        self.start_time = time.time()
        self.memory = 0
        func = [self.A_star,self.UCS,self.IDS,self.BFS,self.DFS][self.names.index(algo_name)]
        actions = func(start,goal)
        return (actions,time.time()-self.start_time,self.memory)
    
    #-------------------------------------------------------------------------------------------
    
    def A_star(self,start, goal):
        def manhattan_distance(puzzle, goal_state):
            puzzle = list(puzzle)
            goal_state = list(goal_state)
            distance = 0
            for i in range(1,9):
                x1, y1 = divmod(i, 3)
                x2, y2 = divmod(goal_state.index(puzzle[i]), 3)
                distance += abs(x1 - x2) + abs(y1 - y2)
            return distance
        def find_blank_square(state):
                for i in range(3):
                    for j in range(3):
                        if state[i][j] == 0:
                            return (i, j)
        def move(state, direction):
            """
            Make a move in the specified direction and return the new state.
            """
            state = np.array(state).reshape((3,3))
            index = np.argwhere(state == 0)
            x, y = index[0][0], index[0][1]
            if direction == 'U':
                if x > 0:
                    state[x][y], state[x-1][y] = state[x-1][y], state[x][y]
                    return state
                else:
                    return None
            if direction == 'D':
                if x < 2:
                    state[x][y], state[x+1][y] = state[x+1][y], state[x][y]
                    return state
                else:
                    return None
            if direction == 'L':
                if y > 0:
                    state[x][y], state[x][y-1] = state[x][y-1], state[x][y]
                    return state
                else:
                    return None
            if direction == 'R':
                if y < 2:
                    state[x][y], state[x][y+1] = state[x][y+1], state[x][y]
                    return state
                else:
                    return None
            return None

        """
        A* algorithm to solve the 8-puzzle problem
        :param start: a 2D numpy array representing the starting state of the puzzle
        :param goal: a 2D numpy array representing the goal state of the puzzle
        :return: a list of actions to solve the puzzle from start to goal, or None if no solution is found
        """
        start = tuple(start.flatten().tolist())
        goal = tuple(goal.flatten().tolist())
        directions = ['U', 'D', 'L', 'R']

        # define a priority queue for the 
        queue = PriorityQueue()
        queue.put((0, start))
        came_from = dict()
        cost_so_far = dict()
        came_from[tuple(start)] = None
        cost_so_far[tuple(start)] = 0

        while not queue.empty():
            self.memory+=1
            if time.time()-self.start_time>60*2:
                return None
            current = queue.get()[1]
            if current == goal:
                break

            for direction in directions:
                next_state = move(current, direction)
                if next_state is None:
                    continue
                next_state = tuple(next_state.flatten().tolist())
                new_cost = cost_so_far[tuple(current)] + 1
                if next_state not in cost_so_far or new_cost < cost_so_far[next_state]:
                    cost_so_far[next_state] = new_cost
                    priority = new_cost + manhattan_distance(next_state, goal)
                    queue.put((priority, next_state))
                    came_from[next_state] = (current, direction)

        # retrace the steps from the goal state to the start state
        if tuple(current) != tuple(goal):
            return None
        actions = []
        while came_from[tuple(current)] is not None:
            current, direction = came_from[tuple(current)]
            actions.append(direction)
        actions.reverse()
        return actions
    #--------------------------------------------------------------------------------------------
    
    def UCS(self,start, goal):
        def get_neighbors(state):
            neighbors = []
            for direction in ['U', 'D', 'L', 'R']:
                neighbor = move_blank_square(state, direction)
                if neighbor is not None:
                    neighbors.append(neighbor)
            return neighbors
        def find_blank_square(state):
            for i in range(3):
                for j in range(3):
                    if state[i][j] == 0:
                        return (i, j)
        def move_blank_square(state, direction):
            row, col = find_blank_square(state)
            if direction == 'U':
                if row == 0:
                    return None
                new_row = row - 1
                new_col = col
            elif direction == 'D':
                if row == 2:
                    return None
                new_row = row + 1
                new_col = col
            elif direction == 'L':
                if col == 0:
                    return None
                new_row = row
                new_col = col - 1
            elif direction == 'R':
                if col == 2:
                    return None
                new_row = row
                new_col = col + 1
            else:
                return None

            new_state = np.copy(state)
            new_state[row][col], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[row][col]
            return new_state

        actions = []
        queue = []
        heapq.heappush(queue, (0, tuple(start.flatten()), actions))
        visited = set()
        while queue:
            if time.time()-self.start_time>60*2:
                return None
            self.memory+=1
            _, state, actions = heapq.heappop(queue)
            state = np.array(state).reshape((3,3))
            if np.array_equal(state, goal):
                return actions
            if tuple(state.flatten()) in visited:
                continue
            visited.add(tuple(state.flatten()))
            for direction in ['U', 'D', 'L', 'R']:
                neighbor = move_blank_square(state, direction)
                if neighbor is not None:
                    cost = len(actions) + 1
                    new_actions = actions + [direction]
                    heapq.heappush(queue, (cost, tuple(neighbor.flatten()), new_actions))
        return None
    #-------------------------------------------------------------------------------------
    def IDS(self,initial_state, goal_state):
        def dls(state, goal_state, limit):
            if state == goal_state:
                return []
            if limit == 0:
                return "cutoff"

            zero_index = state.index(0)
            x, y = divmod(zero_index, 3)

            actions = []
            if x > 0: # up
                up_state = copy.copy(state)
                up_state[zero_index], up_state[zero_index - 3] = up_state[zero_index - 3], up_state[zero_index]
                actions.append((up_state,'U'))
            if x < 2: # down
                down_state = copy.copy(state)
                down_state[zero_index], down_state[zero_index + 3] = down_state[zero_index + 3], down_state[zero_index]
                actions.append((down_state,'D'))
            if y > 0: # left
                left_state = copy.copy(state)
                left_state[zero_index], left_state[zero_index - 1] = left_state[zero_index - 1], left_state[zero_index]
                actions.append((left_state,'L'))
            if y < 2: # right
                right_state = copy.copy(state)
                right_state[zero_index], right_state[zero_index + 1] = right_state[zero_index + 1], right_state[zero_index]
                actions.append((right_state,'R'))

            for action,dirc in actions:
                result = dls(action, goal_state, limit-1)
                if result != "cutoff":
                    return [dirc] + result
            return "cutoff"

        initial_state = list(initial_state.flatten())
        goal_state = list(goal_state.flatten())
        depth = 0
        while True:
            if time.time()-self.start_time>60*2:
                return None
            self.memory+=1
            result = dls(initial_state, goal_state, depth)
            if result != "cutoff":
                return result
            depth += 1
    #-----------------------------------------------------------------------------------------
    def BFS(self,start, goal):
        def bfs_rec(start, goal):
            q = deque([start])
            visited = set()
            while q:
                if time.time()-self.start_time>60*2:
                    return None
                self.memory+=1
                curr = q.popleft()

                if np.array_equal(curr.data, goal):
                    return curr

                visited.add(tuple(curr.data.flatten()))

                row, col = curr.row, curr.col
                moves = [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]
                moves_tag = ['U','D','L','R']
                for move_tag,move in zip(moves_tag,moves):
                    if 0 <= move[0] < 3 and 0 <= move[1] < 3:
                        data = np.copy(curr.data)
                        data[row][col], data[move[0]][move[1]] = data[move[0]][move[1]], data[row][col]

                        if tuple(data.flatten()) not in visited:
                            q.append(State(data, curr, move_tag))

            return None

        result = bfs_rec(State(start, None, None), goal)

        if result is None:
            return None

        actions = []
        while result.parent is not None:
            actions.append(result.move)
            result = result.parent

        actions.reverse()
        return actions
    #-----------------------------------------------------------------------------------------
    def DFS(self,puzzle,goal):
        def dfs_rec(puzzle, path, visited, target):
            self.memory+=1
            if self.memory>2000:
                return None
            if puzzle == target:
                return path

            for i, block in enumerate(puzzle):
                if block == 0:
                    x, y = i // 3, i % 3
                    break

            directions = [('D',1, 0), ('R',0, 1), ('U',-1, 0), ('L',0, -1)]
            for dirc,dx, dy in directions:
                x_new, y_new = x + dx, y + dy
                if x_new >= 0 and x_new < 3 and y_new >= 0 and y_new < 3:
                    puzzle_new = puzzle[:]
                    puzzle_new[x * 3 + y], puzzle_new[x_new * 3 + y_new] = puzzle_new[x_new * 3 + y_new], puzzle_new[x * 3 + y]
                    puzzle_new_tuple = tuple(puzzle_new)
                    if puzzle_new_tuple not in visited:
                        visited.add(puzzle_new_tuple)
                        path_new = path[:]
                        path_new.append(dirc)
                        solution = dfs_rec(puzzle_new, path_new, visited, target)
                        if solution is not None:
                            return solution
            return None

        puzzle = list(puzzle.flatten())
        goal = list(goal.flatten())
        path = dfs_rec(puzzle, [], set(), goal)
        return path

In [78]:
def run_on_algos(start,names=['A*','UCS','IDS','BFS','DFS']):
    start = np.array(start).reshape((3,3))
    algo = Algorithms()
    results = {}
    for name in names:
        path,time,memory = algo.run(name,start)
        results[f'{name}-path']=path
        results[f'{name}-time']=round(time)
        results[f'{name}-memory']=memory
    return results

In [80]:
start = np.array([[1, 5, 2],
                 [4, 0, 3],
                 [7, 8, 6]])

run_on_algos(start)

{'A*-path': ['U', 'R', 'D', 'D'],
 'A*-time': 0,
 'A*-memory': 6,
 'UCS-path': ['U', 'R', 'D', 'D'],
 'UCS-time': 0,
 'UCS-memory': 40,
 'IDS-path': ['U', 'R', 'D', 'D'],
 'IDS-time': 0,
 'IDS-memory': 5,
 'BFS-path': ['U', 'R', 'D', 'D'],
 'BFS-time': 0,
 'BFS-memory': 24,
 'DFS-path': ['D',
  'R',
  'U',
  'U',
  'L',
  'D',
  'D',
  'R',
  'U',
  'U',
  'L',
  'D',
  'D',
  'R',
  'U',
  'U',
  'L',
  'D',
  'D',
  'R',
  'U',
  'U',
  'L',
  'D',
  'D',
  'R'],
 'DFS-time': 0,
 'DFS-memory': 27}

The depth-first search (DFS) algorithm does not guarantee to find the best solution. DFS works by exploring as far as possible along each branch before backtracking. While this approach can sometimes lead to a solution quickly, it can also get stuck in a never-ending loop if the tree is too large or if the goal state is too far down a particular branch.

In other words, DFS is not guaranteed to find the optimal solution because it does not prioritize searching in areas of the tree that are likely to lead to the goal state. Instead, it prioritizes exploring all possible branches as deeply as possible.

In [32]:
f = open('Examples.txt')
s = f.read()
f.close()

In [45]:
clean = lambda x: list(map(int,x.strip().replace(' ','').replace('{','').replace('}','').split(',')))   
examples = [[clean(e2) for e2 in e.strip().split('\n')[1:]] for e in s.split('//')]
examples

[[[1, 2, 3, 0, 7, 6, 5, 4, 8],
  [0, 4, 1, 2, 5, 3, 7, 8, 6],
  [4, 1, 3, 0, 2, 6, 7, 5, 8],
  [1, 2, 3, 0, 4, 8, 7, 6, 5],
  [1, 2, 0, 4, 8, 3, 7, 6, 5],
  [1, 0, 2, 4, 6, 3, 7, 5, 8],
  [0, 1, 2, 4, 5, 3, 7, 8, 6],
  [1, 2, 3, 0, 4, 5, 7, 8, 6],
  [1, 2, 3, 4, 0, 5, 7, 8, 6],
  [1, 2, 3, 4, 5, 0, 7, 8, 6],
  [0, 1, 3, 4, 2, 5, 7, 8, 6],
  [2, 3, 5, 1, 0, 4, 7, 8, 6],
  [1, 6, 2, 5, 3, 0, 4, 7, 8],
  [1, 8, 2, 0, 4, 3, 7, 6, 5],
  [2, 5, 3, 4, 1, 6, 0, 7, 8],
  [1, 2, 3, 4, 6, 8, 7, 5, 0],
  [1, 6, 2, 5, 7, 3, 0, 4, 8],
  [0, 4, 1, 5, 3, 2, 7, 8, 6],
  [0, 5, 2, 1, 8, 3, 4, 7, 6],
  [1, 2, 3, 0, 4, 6, 7, 5, 8]],
 [[1, 3, 5, 7, 2, 6, 8, 0, 4],
  [4, 1, 2, 3, 0, 6, 5, 7, 8],
  [4, 3, 1, 0, 7, 2, 8, 5, 6],
  [5, 2, 1, 4, 8, 3, 7, 6, 0],
  [2, 0, 8, 1, 3, 5, 4, 6, 7],
  [3, 5, 6, 1, 4, 8, 0, 7, 2],
  [1, 0, 2, 7, 5, 4, 8, 6, 3],
  [5, 1, 8, 2, 7, 3, 4, 0, 6],
  [4, 3, 0, 6, 1, 8, 2, 7, 5],
  [2, 4, 3, 1, 6, 5, 8, 0, 7],
  [1, 2, 3, 6, 4, 5, 7, 8, 0],
  [3, 1, 2, 4, 5, 6, 7, 8, 0],
  [1, 2

In [85]:
names=['A*','UCS','IDS','BFS']
dfs = []
for difficulty,i in zip(['easy','medium','hard'],list(range(3))):
    if difficulty=='hard': names.remove('IDS')
    lst = []
    for example in range(len(examples[i])):
        res_dic = run_on_algos(np.array(examples[i][example]),names)
        lst.append(res_dic)
        print('|',end='')
    dfs.append(pd.DataFrame(lst))
    print(f'{difficulty} examples finished!')

||||||||||||||||||||easy examples finished!
|||||||||||||||medium examples finished!
||||||||||||||||||||hard examples finished!


In [89]:
writer = pd.ExcelWriter('file.xlsx')
for name, df in zip(['easy','medium','hard'],dfs):
    df.to_excel(writer,f'{name}')
writer.save() 

In [98]:
pd.concat(dfs).mean()

<ipython-input-98-d6d31b846a60>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.concat(dfs).mean()


A*-time           0.254545
A*-memory      1882.345455
UCS-time          4.200000
UCS-memory    82193.745455
IDS-time         18.600000
IDS-memory       10.800000
BFS-time          5.054545
BFS-memory    77021.545455
dtype: float64

it appears that the A* algorithm performed the best in terms of both time and memory efficiency. It has the fastest running time (0.254545) and low memory usage (1882.345455).

The uniform cost search (UCS) algorithm had a much slower running time (4.2) and higher memory usage (82193.745455) compared to A*.

The iterative deepening search (IDS) algorithm had an even slower running time (18.6) and lower memory usage (10.8) compared to UCS.

The breadth-first search (BFS) algorithm had a slightly slower running time (5.054545) and higher memory usage (77021.545455) compared to UCS.

The reason why A* performs better than the other algorithms is because it uses both the heuristic_cost and the path_cost to determine the priority of each node in the priority queue. This allows A* to prune the search tree more effectively and find the optimal solution more efficiently. Additionally, the heuristic function used in A* helps to guide the search in the direction of the goal state, which can reduce the number of nodes that need to be expanded.

The UCS algorithm does not use a heuristic function and therefore does not have the same level of guidance towards the goal state. This can result in a slower search and higher memory usage as the algorithm has to explore more nodes in the search tree. Similarly, the IDS and BFS algorithms do not use a heuristic function, and they can be even slower and use more memory than UCS as they need to perform multiple full-depth searches in the case of IDS, or explore a large number of nodes in the search tree in the case of BFS.